In [1]:
import pandas as pd
import datetime

In [10]:
legislative_period_19_begin = datetime.date(2017, 10, 24)
legislative_period_19_end = datetime.date(2021, 10, 26)

In [11]:
#read xml file to pandas dataframe
df = pd.read_xml('..\\data\\MDB_STAMMDATEN.xml', parser="lxml",xpath='//MDB[WAHLPERIODEN//WAHLPERIODE/WP=19]//NAMEN/NAME')
#remove row where HISTORIE_BIS is not empty (these are duplicates)
df = df[df['HISTORIE_BIS'].isnull()]
#add complete name column
df['NAME'] = df['ANREDE_TITEL'] + ' ' + df['VORNAME'] + ' ' + df['NACHNAME']
df['NAME'] = df['NAME'].fillna(df['VORNAME'] + ' ' + df['NACHNAME'])
#remove columns that are not needed
df = df.drop(['ORTSZUSATZ','ADEL','PRAEFIX','HISTORIE_VON','HISTORIE_BIS'],axis=1)
# add columns for legislative period
df['START'] = [legislative_period_19_begin]*len(df)
df['END'] = [legislative_period_19_end]*len(df) 

In [12]:
df

,NACHNAME,VORNAME,ANREDE_TITEL,AKAD_TITEL,NAME,START,END
0,Bleser,Peter,None,None,Peter Bleser,2017-10-24,2021-10-26
1,Brecht,Eberhard,Dr.,Dr.,Dr. Eberhard Brecht,2017-10-24,2021-10-26
3,Dehm,Diether,Dr.,Dr.,Dr. Diether Dehm,2017-10-24,2021-10-26
4,Fuchtel,Hans-Joachim,None,None,Hans-Joachim Fuchtel,2017-10-24,2021-10-26
5,Gysi,Gregor,Dr.,Dr.,Dr. Gregor Gysi,2017-10-24,2021-10-26
...,...,...,...,...,...,...,...
785,Emmerich,Marcel,None,None,Marcel Emmerich,2017-10-24,2021-10-26
786,Wetterich,Susanne,None,None,Susanne Wetterich,2017-10-24,2021-10-26
787,Jäger,Florian,None,None,Florian Jäger,2017-10-24,2021-10-26
788,Gökhan,Zeki,None,None,Zeki Gökhan,2017-10-24,2021-10-26


In [13]:
#read excel sheet with MPs that left the Bundestag before the end of the legislative period
exes = pd.read_excel('..\\data\\Ausgeschiedene.xlsx')

#in column 'Abgeordnete/r' and 'Nachfolger/in' remove everything after the first comma
exes['Nachfolger/in'] = exes['Nachfolger/in'].str.split(',').str[0]
exes['Abgeordnete/r'] = exes['Abgeordnete/r'].str.split(',').str[0]

#in columns 'Abgeordnete/r' and 'Nachfolger/in' remove everything after the first bracket
exes['Nachfolger/in'] = exes['Nachfolger/in'].str.split('(').str[0]
exes['Abgeordnete/r'] = exes['Abgeordnete/r'].str.split('(').str[0]

In [14]:
exes.shape

(41, 4)

In [15]:
# for the MPs who left/joined parliament during the 19th election period, change start/end date in df

#get index of df where 'name' is the same as x
def get_index(x):
    return df[df['NAME'] == x].index.values.astype(int)


# match all the names from all mps with the ones that left earlier/joined later and change start/end date
for r in exes.index:
    idx = get_index(exes['Abgeordnete/r'][r])
    if len(idx) == 0:
        print(exes['Abgeordnete/r'][r])
    df.loc[idx,'END'] = exes['ausgeschieden'][r]
    idx = get_index(exes['Nachfolger/in'][r])
    if len(idx)==0:
        print(exes['Nachfolger/in'][r])
    df.loc[idx,'START'] = exes['eingetreten'][r]


Prof. Dr. Axel Gehrke
Axel Troost
Stephan Kühn 
Reginald Hanke
Dr. Ursula von der Leyen
Isabel Mackensen


In [16]:
df[df['START']!=legislative_period_19_begin]

,NACHNAME,VORNAME,ANREDE_TITEL,AKAD_TITEL,NAME,START,END
1,Brecht,Eberhard,Dr.,Dr.,Dr. Eberhard Brecht,2019-10-25 00:00:00,2021-10-26
38,Siebert,Bernd,None,None,Bernd Siebert,2020-03-02 00:00:00,2021-10-26
176,Rawert,Mechthild,None,None,Mechthild Rawert,2020-05-26 00:00:00,2021-10-26
354,Fabritius,Bernd,Dr. Dr. h. c.,Dr. Dr. h. c.,Dr. Dr. h. c. Bernd Fabritius,2021-03-22 00:00:00,2021-10-26
361,Freudenstein,Astrid,Dr.,Dr.,Dr. Astrid Freudenstein,2019-07-02 00:00:00,2020-05-15 00:00:00
403,Lotze,Hiltrud,None,None,Hiltrud Lotze,2020-11-24 00:00:00,2021-10-26
405,Manderla,Gisela,None,None,Gisela Manderla,2018-11-05 00:00:00,2021-10-26
421,Ostermann,Tim,Dr.,Dr.,Dr. Tim Ostermann,2021-05-01 00:00:00,2021-10-26
423,Pahlmann,Ingrid,None,None,Ingrid Pahlmann,2019-08-01 00:00:00,2021-10-26
425,Paschke,Markus,None,None,Markus Paschke,2019-11-04 00:00:00,2021-10-26


In [17]:
df.to_excel('..//data//MPs_WP19.xlsx')

In [ ]:
#I manually changed the dates for the MPs who could not be matched automatically (see excel file) !!!!!!